# Análise de texto de fontes desestruturadas e Web

## Aula 11

Nesta aula vamos fazer uma aplicação prática de Machine Learning.

A biblioteca utilizada será a **scikit-learn**.

Para conhecer mais sobre ela, acesse https://scikit-learn.org/stable/tutorial/basic/tutorial.html

## Importando as bibliotecas necessárias

Agora, vamos importar as bibliotecas necessárias:

In [5]:
# para trabalhar com diretórios / sistema operacional
import os

# para trabalhar com expressões regulares
import re

# utilizada para nos indicar o caminho do executável do Python
import sys

# para pandas DataFrame
import pandas as pd

# ML
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Pipeline
from sklearn.pipeline import Pipeline

Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!

Você pode conferir de onde está executando o Python e qual a versão

In [2]:
print('Executável:')
print(sys.executable)

print('\nVersão do Python:')
print(sys.version)

Executável:
/opt/conda/bin/python

Versão do Python:
3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]


Vamos conferir em qual diretório iremos trabalhar (é o diretório do notebook)

In [3]:
print('O seu notebook está na pasta:')
print(os.getcwd())

O seu notebook está na pasta:
/home/jovyan


# Text Vectorization

Vamos aprender como transformar textos de forma a conseguir treinar modelos a partir deles utilizando o modelo **Bag of Words (BoW)**. No python, o BoW já está implementado como CountVectorizer na biblioteca **sklearn**.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Considere o dataframe de exemplo

In [4]:
df = pd.DataFrame({'Texto': ['quero ir para Praia', 'não gosto de praia, só de sol', 'quero sombra e água']})
df

,Texto
0,quero ir para Praia
1,"não gosto de praia, só de sol"
2,quero sombra e água


Primeiro, criamos um CountVectorizer

In [6]:
exvec = CountVectorizer(binary=True)

E mandamos ele se *ajustar* aos dados, ou seja, aprender o vocabulário (fit) e já devoltar os dados de treinamento transformados (transform).

In [25]:
mat = exvec.fit_transform(df["Texto"]).toarray()
mat

array([[0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]])

In [26]:
exvec.get_feature_names_out()

array(['de', 'gosto', 'ir', 'não', 'para', 'praia', 'quero', 'sol',
       'sombra', 'só', 'água'], dtype=object)

In [27]:
df_vec = pd.DataFrame(mat)
df_vec.columns = exvec.get_feature_names_out()
df_vec

,de,gosto,ir,não,para,praia,quero,sol,sombra,só,água
0,0,0,1,0,1,1,1,0,0,0,0
1,1,1,0,1,0,1,0,1,0,1,0
2,0,0,0,0,0,0,1,0,1,0,1


### Utilizando com dados novos

Agora, podemos utilizar o CountVectorizer com dados novos (frases ainda não vistas).

In [28]:
df

,Texto
0,quero ir para Praia
1,"não gosto de praia, só de sol"
2,quero sombra e água


Primeiro exemplo utilizando uma frase que já pertencia a base

In [29]:
txt = "quero sombra e água"
exvec.transform([txt]).toarray()

array([[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]])

Modificando um pouco a frase

In [30]:
txt = "em copacabana quero sombra e água"
exvec.transform([txt]).toarray()

array([[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]])

Por que ele não modificou?

R:

# Notícias
## Carregando os dados

Vamos abrir os dados que estão armazenados em planilhas. Você pode escolher a versão CSV ou XLSX

In [6]:
df = pd.read_csv('notebooks/lessons/11/noticias.csv')

Caso prefira utilizar a versão Excel, abra com

In [ ]:
df = pd.read_excel('noticias.xlsx')

Vamos ver o que temos armazenado no DataFrame?!

In [33]:
df.head(10)

,Secao,Titulo,Descrição,Data
0,politica,Novas parcelas do auxílio emergencial ganham f...,A possibilidade de convocação de sessão legisl...,13/01/2021 09:08
1,economia,Café arábica recua de máxima de 4 anos na ICE;...,NOVA YORK/LONDRES (Reuters) – Os contratos fut...,07/05/2021 19:09
2,economia,Milho e soja tocam máximas de vários anos em C...,Por Tom Polansek CHICAGO (Reuters) – Preocupaç...,07/05/2021 18:11
3,tecnologia,Brasil termina abril com 82.000 mortes por cov...,O Brasil somou 2.595 mortes por covid-19 nesta...,30/04/2021 20:16
4,tecnologia,Vai comprar como?,Mesmo com a aceleração da digitalização durant...,06/05/2021 18:30
5,politica,"‘Há uma grande operação abafa em curso’, diz e...","PEC da Blindagem, derrubada da quebra do sigil...",27/02/2021 15:15
6,politica,Conheça os projetos que propõem prorrogar o au...,O Congresso Nacional já recebeu vários projeto...,25/01/2021 18:08
7,tecnologia,A reestruturação do SAS,"Promovido a vice-presidente nas Américas, Marv...",06/05/2021 18:30
8,politica,TSE mantém protocolos sanitários no segundo turno,O Tribunal Superior Eleitoral (TSE) mantém hoj...,29/11/2020 07:10
9,tecnologia,"Laboratório Moderna, pioneiro da vacina antico...","O laboratório Moderna, pioneiro nas vacinas co...",29/04/2021 08:36


Quantas notícias temos ao todo?

In [34]:
df.shape

(600, 4)

Quantas seções de notícias temos? E quantas notícias por seção?

In [35]:
df['Secao'].value_counts()

Secao
politica      200
economia      200
tecnologia    200
Name: count, dtype: int64

# Separando em Treino e Teste

Para simular uma situação real de uso do classificador e termos uma ideia de como o classificador se sairá em um uso futuro, iremos dividir nossa base de dados em duas, utilizando uma base para treinar o modelo e outra base para avaliar o desempenho do modelo treinado.

In [7]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(df[['Titulo', 'Descrição']],
                                                            df['Secao'],
                                                            test_size=0.25,
                                                            random_state=151)

Conferindo os dados de treinamento (X e y)

In [37]:
X_train_raw

,Titulo,Descrição
180,Pacheco descarta ‘imposto temporário’ para ban...,"O presidente do Senado, Rodrigo Pacheco (DEM-M..."
98,Bolsonaro ergue cartaz ‘Globo lixo’ em aeropor...,O presidente Jair Bolsonaro ergueu um cartaz o...
242,Casos de Covid da Índia ficam próximos de alta...,NOVA DÉLHI (Reuters) – As infecções e mortes d...
72,Plantio de milho dos EUA atinge 67%; soja vai ...,Por Julie Ingwersen CHICAGO (Reuters) – O Depa...
546,Italiana recebe seis doses da vacina da Pfizer...,Enquanto milhares de pessoas aguardam a tão es...
...,...,...
85,"Fernando Azevedo e Silva, ministro da Defesa, ...","Na tarde desta segunda-feira (29), o ministro ..."
71,"EUA/Fed: crédito ao consumidor cresce US$ 25,8...",O crédito ao consumidor dos Estados Unidos cre...
556,Obesidade entre jovens amplia potencial de ris...,Um estudo feito pelas universidades de Oxford ...
373,"Selic no fim de 2021 permanece em 5,50% ao ano...",Os economistas do mercado financeiro mantivera...


In [38]:
y_train

180      politica
98       politica
242      economia
72       economia
546      economia
          ...    
85       politica
71       economia
556    tecnologia
373      economia
408      politica
Name: Secao, Length: 450, dtype: object

## Extração de Features

Não vamos utilizar direto o título para treino de um modelo. A partir do título, iremos criar variáveis com o uso da técnica de **Bag of Words**.

Primeiro, criamos um CountVectorizer

In [39]:
cvec = CountVectorizer()

E *ajustar* ele aos dados, ou seja, fazer aprender o vocabulário (fit) e já devoltar os dados de treinamento transformados (transform)

In [40]:
mat = cvec.fit_transform(X_train_raw['Titulo']).toarray()

In [41]:
X_train = pd.DataFrame(mat)
X_train.columns = cvec.get_feature_names_out()
X_train.sample(20)

,000,04,06,10,100,108mp,11,12,135,14,...,zuckerberg,às,áfrica,área,áudio,áustria,índia,índico,órgão,única
197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
163,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Agora com os dados de teste

In [42]:
X_test_raw

,Titulo,Descrição
552,Grande estudo confirma eficácia da vacina cont...,A vacina Pfizer-BioNTech é mais de 95% eficaz ...
391,Anvisa defende decisão de vetar compra da vaci...,A Agência Nacional de Vigilância Sanitária (An...
547,Doria vai para Miami e vice-governador testa p...,"Ontem (22), o governador de São Paulo, João Do..."
271,Tesla arrecada US$ 50 milhões em reservas para...,A Tesla conseguiu arrecadar mais de US$ 50 mil...
425,“O silêncio não é uma opção” para os palestino...,"Adnan, um palestino de Jerusalém Oriental, pro..."
...,...,...
538,Vendas de soja da safra 2021/22 em Mato Grosso...,Por Nayara Figueiredo SÃO PAULO (Reuters) – A ...
554,Nova cepa do coronavírus encontrada no Rio est...,A Secretaria de Estado de Saúde (SES) do Rio d...
478,Segundo turno: 5 dicas do que fazer antes de v...,O segundo turno das eleições municipais está c...
598,"Dólar pausa quedas, mas segue perto de mínimas...",Por José de Castro SÃO PAULO (Reuters) – O dól...


In [43]:
mat_test = cvec.transform(X_test_raw['Titulo']).toarray()

In [44]:
X_test = pd.DataFrame(mat_test)
X_test.columns = cvec.get_feature_names_out()
X_test.sample(2)

,000,04,06,10,100,108mp,11,12,135,14,...,zuckerberg,às,áfrica,área,áudio,áustria,índia,índico,órgão,única
88,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
114,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
X_test.head(2)

,000,04,06,10,100,108mp,11,12,135,14,...,zuckerberg,às,áfrica,área,áudio,áustria,índia,índico,órgão,única
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Criando o classificador
Primeiro, vamos utilizar um **DecisionTreeClassifier**

In [46]:
dt = DecisionTreeClassifier()

Para treiná-lo, basta fazer:

In [47]:
dt.fit(X_train, y_train)

DecisionTreeClassifier()

## Base de testes
Agora, vamos ver a performance na base de testes

In [48]:
X_test.shape

(150, 1981)

In [49]:
y_pred = dt.predict(X_test)
accuracy_score(y_true=y_test, y_pred=y_pred)

0.5866666666666667

## Utilizando Pipeline
Agora, vamos ver a performance na base de testes

In [50]:
pipe = Pipeline([('vect', CountVectorizer(binary=True)),
                 ('rf', RandomForestClassifier(n_estimators=100, n_jobs=-1))])

pipe.fit(X_train_raw['Descrição'], y_train)

Pipeline(steps=[('vect', CountVectorizer(binary=True)),
                ('rf', RandomForestClassifier(n_jobs=-1))])

Performance nos dados de treinamento:

In [51]:
y_pred = pipe.predict(X_train_raw['Descrição'])
accuracy_score(y_true=y_train, y_pred=y_pred)

0.9866666666666667

Performance nos dados de teste:

In [52]:
y_pred = pipe.predict(X_test_raw['Descrição'])
accuracy_score(y_true=y_test, y_pred=y_pred)

0.7266666666666667

# Exercícios

**Exercício 1)** Veja na documentação do `CountVectorizer` o parâmetro `stop_words`. Passe uma lista de palavras para este parâmetro. Explique o que acontece.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Dica: procure no Google por `nltk stopwords portuguese`

In [3]:
!pip install nltk
!python -m nltk.downloader stopwords

/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
from nltk.corpus import stopwords


pipe = Pipeline([
    ('vect', CountVectorizer(
        binary=True,
        stop_words=stopwords.words('portuguese')
    )),
    ('rf', RandomForestClassifier(
        n_estimators=100,
        n_jobs=-1
    ))
])

pipe.fit(X_train_raw['Descrição'], y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(binary=True,
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('rf', RandomForestClassifier(n_jobs=-1))])

In [9]:
y_pred = pipe.predict(X_train_raw['Descrição'])
accuracy_score(y_true=y_train, y_pred=y_pred)

0.9866666666666667

In [10]:
y_pred = pipe.predict(X_test_raw['Descrição'])
accuracy_score(y_true=y_test, y_pred=y_pred)

0.78

R:

**Exercício 2)** Veja na documentação do CountVectorizer o parâmetro `vocabulary`. Passe uma lista de palavras para este parâmetro. Explique o que acontece.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

**Exercício 3)** Veja na documentação do CountVectorizer o parâmetro `binary`. O que acontece se treinarmos um CountVectorizer com `binary=False`? Que impactos, positivos ou negativos, isto pode trazer ao nosso modelo?

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

**Exercício 4)** Considerando a versão atualizada do seu `CountVectorizer`, verifique a performance de outros modelos. Algumas sugestões:

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

**Exercício 5)** No `CountVectorizer`, consideramos apenas as palavras isoladamente. Entretanto, a ocorrência de certas palavras após outra determinada palavra também é algo informativo. Confira o parâmetro `ngram_range` na documentação do `CountVectorizer`. Teste algumas variações para o `ngram_range` e analise os resultados.

**Exercício 6)** Existem outras formas de implementar um Vectorizer. Uma opção bastante utilizada é o `tf-idf`.

Confira tutoriais em:

http://www.tfidf.com

https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76

E a documentação oficial do sk-lean:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Então, utilize o `TfidfVectorizer` para vetorização dos testes e avalie sua performance.

**Exercício 7)** Os modelos baseados em árvore também podem ser utilizados para seleção de features. Treine um `DecisionTreeClassifier` utilizando uma profundidade baixa (Ex: 4). Desenhe a árvore (aula 09) e confira quais features foram utilizadas para separação das amostras.